In [5]:
import datasets
import transformers

from rich import print

In [2]:
DEBUG = True
MODEL = "Helsinki-NLP/opus-mt-en-ro"
SEQ_LENGTH = 32
BATCH_SIZE = 8
PERCENT = 5

In [6]:
# dataset = datasets.load_dataset('imdb', split='train[:5%]')  # ['train', 'test', 'unsupervised']
dataset = datasets.load_dataset('wmt16', 'de-en')
metric = datasets.load_metric('sacrebleu')

Reusing dataset wmt16 (/home/tiankang/.cache/huggingface/datasets/wmt16/de-en/1.0.0/0d9fb3e814712c785176ad8cdb9f465fbe6479000ee6546725db30ad8a8b5f8a)


In [7]:
datasets

<module 'datasets' from '/home/tiankang/software/anaconda3/lib/python3.8/site-packages/datasets/__init__.py'>

In [9]:
print(dataset['train'][0])

{
    'translation': {
        'de': 'Wiederaufnahme der Sitzungsperiode',
        'en': 'Resumption of the session'
    }
}

In [13]:
print(metric)

Metric(name: "sacrebleu", features: {'predictions': Value(dtype='string', id='sequence'), 
'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, 
id='references')}, usage: """
Produces BLEU scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions: The system stream (a sequence of segments)
    references: A list of one or more reference streams (each a sequence of segments)
    smooth: The smoothing method to use
    smooth_value: For 'floor' smoothing, the floor to use
    force: Ignore data that looks already tokenized
    lowercase: Lowercase the data
    tokenize: The tokenizer to use
Returns:
    'score': BLEU score,
    'counts': Counts,
    'totals': Totals,
    'precisions': Precisions,
    'bp': Brevity penalty,
    'sys_len': predictions length,
    'ref_len': reference length,
Examples:

    >>> predictions = ["hello there general kenobi", "foo bar foobar"]
    >>> references = [["hello there general kenobi", "hello there !"], ["foo bar foobar", 
"foo bar foobar"]]
    >>> sacrebleu = datasets.load_metric("sacrebleu")
    >>> results = sacrebleu.compute(predictions=predictions, references=references)
    >>> print(list(results.keys()))
    ['score', 'counts', 'totals', 'precisions', 'bp', 'sys_len', 'ref_len']
    >>> print(round(results["score"], 1))
    100.0
""", stored examples: 0)

In [17]:
fake_preds = ["hello there general kenobi star war", "foo bar foobar"]
fake_labels = [["hello there general kenobi star war", "hello there !"], ["foo bar foobar", 
"foo bar foobar"]]
metric.compute(predictions=fake_preds, references=fake_labels)

{'score': 100.00000000000004,
 'counts': [9, 7, 5, 3],
 'totals': [9, 7, 5, 3],
 'precisions': [100.0, 100.0, 100.0, 100.0],
 'bp': 1.0,
 'sys_len': 9,
 'ref_len': 9}

In [23]:
tokenizer = transformers.AutoTokenizer.from_pretrained(MODEL, cache_dir='/home/tiankang/wusuowei/data/huggingface/tokenizers')

In [24]:
if 'mbart' in MODEL:
    tokenizer.src_lang = 'en-XX'
    tokenizer.tgt_lang = 'ro-RO'

In [25]:
print(tokenizer)

PreTrainedTokenizer(name_or_path='Helsinki-NLP/opus-mt-en-ro', vocab_size=59543, 
model_max_len=512, is_fast=False, padding_side='right', special_tokens={'eos_token': '</s>', 
'unk_token': '<unk>', 'pad_token': '<pad>'})

In [31]:
print(tokenizer.tokenize("Hello, this one sentence!"))
print(tokenizer.encode("Hello, this one sentence!"))
print(tokenizer("Hello, this one sentence!"))

with tokenizer.as_target_tokenizer():
    # This context manager will make sure the tokenizer uses the special tokens corresponding to the targets.
    print(tokenizer.tokenize("Hello, this one sentence!"))
    print(tokenizer("Hello, this one sentence!"))

['▁He', 'llo', ',', '▁this', '▁one', '▁sentence', '!']

[125, 778, 3, 63, 141, 9191, 23, 0]

{'input_ids': [125, 778, 3, 63, 141, 9191, 23, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 
1]}

['▁Hel', 'lo', ',', '▁', 'this', '▁o', 'ne', '▁se', 'nten', 'ce', '!']

{'input_ids': [10334, 1204, 3, 15, 8915, 27, 452, 59, 29579, 581, 23, 0], 'attention_mask': 
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [26]:
with tokenizer.as_target_tokenizer():
    # This context manager will make sure the tokenizer uses the special tokens corresponding to the targets.
    print(tokenizer(["Hello, this one sentence!", "This is another sentence."]))

{'input_ids': [[10334, 1204, 3, 15, 8915, 27, 452, 59, 29579, 581, 23, 0], [235, 1705, 11, 
32, 8, 1205, 5305, 59, 29579, 581, 2, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 
1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [33]:
if MODEL in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "translate English to Romanian: "
else:
    prefix = ""

max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "ro"

def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    # tokenizer for source
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    # tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [34]:
print(preprocess_function(raw_datasets['train'][:2]))

{'input_ids': [[393, 4462, 14, 1137, 53, 216, 28636, 0], [24385, 14, 28636, 14, 4646, 4622, 
53, 216, 28636, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1,
1]], 'labels': [[42140, 494, 1750, 53, 8, 59, 903, 3543, 9, 15202, 0], [36199, 6612, 9, 
15202, 122, 568, 35788, 21549, 53, 8, 59, 903, 3543, 9, 15202, 0]]}

In [35]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

In [38]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(MODEL)

ModuleNotFoundError: No module named 'torch_scatter'

In [8]:
dataset[0]

{'label': 1,
 'text': 'Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High\'s satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers\' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I\'m here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn\'t!'}

In [9]:
tokenizer = transformers.BertTokenizerFast.from_pretrained(MODEL)

In [10]:
tokenizer.encode(dataset[0]['text'])

[101,
 22953,
 2213,
 4381,
 2152,
 2003,
 1037,
 9476,
 4038,
 1012,
 2009,
 2743,
 2012,
 1996,
 2168,
 2051,
 2004,
 2070,
 2060,
 3454,
 2055,
 2082,
 2166,
 1010,
 2107,
 2004,
 1000,
 5089,
 1000,
 1012,
 2026,
 3486,
 2086,
 1999,
 1996,
 4252,
 9518,
 2599,
 2033,
 2000,
 2903,
 2008,
 22953,
 2213,
 4381,
 2152,
 1005,
 1055,
 18312,
 2003,
 2172,
 3553,
 2000,
 4507,
 2084,
 2003,
 1000,
 5089,
 1000,
 1012,
 1996,
 25740,
 2000,
 5788,
 13732,
 1010,
 1996,
 12369,
 3993,
 2493,
 2040,
 2064,
 2156,
 2157,
 2083,
 2037,
 17203,
 5089,
 1005,
 13433,
 8737,
 1010,
 1996,
 9004,
 10196,
 4757,
 1997,
 1996,
 2878,
 3663,
 1010,
 2035,
 10825,
 2033,
 1997,
 1996,
 2816,
 1045,
 2354,
 1998,
 2037,
 2493,
 1012,
 2043,
 1045,
 2387,
 1996,
 2792,
 1999,
 2029,
 1037,
 3076,
 8385,
 2699,
 2000,
 6402,
 2091,
 1996,
 2082,
 1010,
 1045,
 3202,
 7383,
 1012,
 1012,
 1012,
 1012,
 1012,
 1012,
 1012,
 1012,
 1012,
 2012,
 1012,
 1012,
 1012,
 1012,
 1012,
 1012,
 1012,
 1012,
 101

In [11]:
tokenizer.tokenize(dataset[0]['text'])

['bro',
 '##m',
 '##well',
 'high',
 'is',
 'a',
 'cartoon',
 'comedy',
 '.',
 'it',
 'ran',
 'at',
 'the',
 'same',
 'time',
 'as',
 'some',
 'other',
 'programs',
 'about',
 'school',
 'life',
 ',',
 'such',
 'as',
 '"',
 'teachers',
 '"',
 '.',
 'my',
 '35',
 'years',
 'in',
 'the',
 'teaching',
 'profession',
 'lead',
 'me',
 'to',
 'believe',
 'that',
 'bro',
 '##m',
 '##well',
 'high',
 "'",
 's',
 'satire',
 'is',
 'much',
 'closer',
 'to',
 'reality',
 'than',
 'is',
 '"',
 'teachers',
 '"',
 '.',
 'the',
 'scramble',
 'to',
 'survive',
 'financially',
 ',',
 'the',
 'insight',
 '##ful',
 'students',
 'who',
 'can',
 'see',
 'right',
 'through',
 'their',
 'pathetic',
 'teachers',
 "'",
 'po',
 '##mp',
 ',',
 'the',
 'pet',
 '##tine',
 '##ss',
 'of',
 'the',
 'whole',
 'situation',
 ',',
 'all',
 'remind',
 'me',
 'of',
 'the',
 'schools',
 'i',
 'knew',
 'and',
 'their',
 'students',
 '.',
 'when',
 'i',
 'saw',
 'the',
 'episode',
 'in',
 'which',
 'a',
 'student',
 'repeated

In [12]:
tokenizer(dataset[0]['text'], return_tensors='pt')

{'input_ids': tensor([[  101, 22953,  2213,  4381,  2152,  2003,  1037,  9476,  4038,  1012,
          2009,  2743,  2012,  1996,  2168,  2051,  2004,  2070,  2060,  3454,
          2055,  2082,  2166,  1010,  2107,  2004,  1000,  5089,  1000,  1012,
          2026,  3486,  2086,  1999,  1996,  4252,  9518,  2599,  2033,  2000,
          2903,  2008, 22953,  2213,  4381,  2152,  1005,  1055, 18312,  2003,
          2172,  3553,  2000,  4507,  2084,  2003,  1000,  5089,  1000,  1012,
          1996, 25740,  2000,  5788, 13732,  1010,  1996, 12369,  3993,  2493,
          2040,  2064,  2156,  2157,  2083,  2037, 17203,  5089,  1005, 13433,
          8737,  1010,  1996,  9004, 10196,  4757,  1997,  1996,  2878,  3663,
          1010,  2035, 10825,  2033,  1997,  1996,  2816,  1045,  2354,  1998,
          2037,  2493,  1012,  2043,  1045,  2387,  1996,  2792,  1999,  2029,
          1037,  3076,  8385,  2699,  2000,  6402,  2091,  1996,  2082,  1010,
          1045,  3202,  7383,  1012,  

In [6]:
tokenizer = transformers.BertTokenizerFast.from_pretrained(MODEL)

In [13]:
def tokenize(x):
    x['input_ids'] = tokenizer.encode(
        x['text'],
        max_length=SEQ_LENGTH,
        truncation=True,
        # pad_to_max_length=True,
        padding='max_length'
    )
    return x

def prepare_dataset(split):
    split_pct = BATCH_SIZE if DEBUG else PERCENT
    dataset = nlp.load_dataset('imdb', split=f'{split}[:{split_pct}%]')
    dataset = dataset.map(tokenize, batched=True)
    dataset.set_format(type='torch', columns=['input_ids', 'label'])
    return dataset

In [14]:
train_dataset = dataset.map(tokenize)

In [15]:
train_dataset[0]

{'input_ids': [101,
  22953,
  2213,
  4381,
  2152,
  2003,
  1037,
  9476,
  4038,
  1012,
  2009,
  2743,
  2012,
  1996,
  2168,
  2051,
  2004,
  2070,
  2060,
  3454,
  2055,
  2082,
  2166,
  1010,
  2107,
  2004,
  1000,
  5089,
  1000,
  1012,
  2026,
  102],
 'label': 1,
 'text': 'Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High\'s satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers\' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I\'m here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I 

In [16]:
dataset

Dataset(features: {'text': Value(dtype='string', id=None), 'label': ClassLabel(num_classes=2, names=['neg', 'pos'], names_file=None, id=None)}, num_rows: 1250)

In [17]:
train_dataset

Dataset(features: {'input_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'label': ClassLabel(num_classes=2, names=['neg', 'pos'], names_file=None, id=None), 'text': Value(dtype='string', id=None)}, num_rows: 1250)

In [18]:
train_dataset.set_format(type='torch', columns=['input_ids', 'label'])  #output_all_columns=True)
train_dataset

Dataset(features: {'input_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'label': ClassLabel(num_classes=2, names=['neg', 'pos'], names_file=None, id=None), 'text': Value(dtype='string', id=None)}, num_rows: 1250)

In [19]:
train_dataset[0]

/home/tiankang/software/anaconda3/lib/python3.8/site-packages/nlp/utils/py_utils.py:191: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return function(data_struct)


{'input_ids': tensor([  101, 22953,  2213,  4381,  2152,  2003,  1037,  9476,  4038,  1012,
          2009,  2743,  2012,  1996,  2168,  2051,  2004,  2070,  2060,  3454,
          2055,  2082,  2166,  1010,  2107,  2004,  1000,  5089,  1000,  1012,
          2026,   102]),
 'label': tensor(1)}